In [74]:
from openai import OpenAI
from g4f.client import Client
from db_utils import csv_read,query_database
import re
import pandas as pd

In [75]:
client = Client()

In [76]:
def generate_sql_from_nl(user_input):
    csv_string = csv_read()
    system_prompt = f''''
        1. 你是一個資料庫專家，能夠生成相應的SQL指令。 
        2. SQL的編寫請用全英文。
        3. 非常重要!返回sql語法即可，不用其他說明與文字。
        4. MYSQL code中請不要出現沒有的英文名稱。
        '''

    user_prompt = f'''
        資料庫結構、表名稱、欄位名稱請參考以下內容:{csv_string}
                    '''
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input + user_prompt}
        ],
        max_tokens=150,
        temperature=0.1
    )
    sql_response = response.choices[0].message.content
    return sql_response


In [77]:
def extract_sql_query(sql_response): 
    match = re.search(r'SELECT.*', sql_response, re.IGNORECASE) 
    if match: 
        return match.group() 
    else: 
        raise ValueError("SQL查詢指令未找到")

In [78]:
if __name__ == "__main__":
    user_input ="請幫我查詢員工的姓名、編號資料"   #input("請輸入內容: ") 
    # 初次調用GPT判斷是否需要function 
    response_1 = client.chat.completions.create( model="gpt-4o-mini", 
                                                messages=[
                                                {"role": "system", 
                                                 "content":"請判斷使用者輸入是否需要連線到資料庫查詢，若需要請回答YES即可!若不需要則正常對話"},
                                                {"role": "user", "content": user_input}])
    
    print('===== 初次調用GPT判斷 =====')
    if 'YES' in response_1.choices[0].message.content:
        sql_response = generate_sql_from_nl(user_input) 
        sql_query = extract_sql_query(sql_response)
        print(f'生成的SQL指令: {sql_query}') 
        # 執行GPT生成的SQL查詢 
        result = query_database(sql_query)
        print(f'查詢結果: {result}')

    else: 
        # 一般對話處理
        print(response_1.choices[0].message.content )
         


===== 初次調用GPT判斷 =====
生成的SQL指令: SELECT ENAME, EMPNO FROM emp;
連線成功
查詢結果: [{"ENAME": "ADAMS", "EMPNO": 7876}, {"ENAME": "ALLEN", "EMPNO": 7499}, {"ENAME": "BLAKE", "EMPNO": 7698}, {"ENAME": "CLARK", "EMPNO": 7782}, {"ENAME": "FORD", "EMPNO": 7902}, {"ENAME": "JAMES", "EMPNO": 7900}, {"ENAME": "JONES", "EMPNO": 7566}, {"ENAME": "KING", "EMPNO": 7839}, {"ENAME": "MARTIN", "EMPNO": 7654}, {"ENAME": "MILLER", "EMPNO": 7934}, {"ENAME": "SCOTT", "EMPNO": 7788}, {"ENAME": "SMITH", "EMPNO": 7369}, {"ENAME": "TURNER", "EMPNO": 7844}, {"ENAME": "WARD", "EMPNO": 7521}]


C:\Users\TMP214\AppData\Local\Programs\Python\Python39\lib\asyncio\events.py:80: UserWarning: Curlm alread closed! quitting from process_data
  self._context.run(self._callback, *self._args)
